In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

In [3]:
import os
import sys
module_path = os.path.join(os.getcwd(), '..')
sys.path.append(module_path)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "deepset/roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

In [5]:
from lrp_engine import LRPEngine, checkpoint_hook
from lrp_engine.lrp_graph import make_graph

In [6]:
lrp = LRPEngine(topk=1, use_gamma=True)

In [7]:
context = "Welcome to the final examination for this term's offering of CS100. Please remove all headphones and earbuds, as well as hats and hoods. Place your bag under your desk so that it does not block the aisle. You are permitted writing instruments, a clear water bottle, and any aids listed on the front of your booklet. The exam will be 150 minutes in duration. You may now begin."
question = "What is this?"

In [9]:
input_ids = tokenizer(question, context, return_tensors="pt")["input_ids"]

In [10]:
output = model(input_ids.to(device))

In [13]:
g = make_graph(output.start_logits)
g[2]

{'AccumulateGrad',
 'AddBackward0',
 'CloneBackward0',
 'CopySlices',
 'EmbeddingBackward0',
 'GeluBackward0',
 'MmBackward0',
 'NativeLayerNormBackward0',
 'PermuteBackward0',
 'ScaledDotProductEfficientAttentionBackward0',
 'SplitBackward0',
 'SqueezeBackward1',
 'TBackward0',
 'TransposeBackward0',
 'ViewBackward0'}

In [11]:
start = torch.argmax(output.start_logits)
end = torch.argmax(output.end_logits) + 1

answer_tokens = input_ids[0][start:end]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
print(answer)

 final examination


# Dummy Test (Don't run if you want to run the full SQuADv2 evaluation below)

In [12]:
checkpoint_vals, param_vals = lrp.run((output.start_logits, output.end_logits))

In [13]:
checkpoint_vals1, param_vals1 = lrp.run((output.start_logits, output.end_logits))

In [14]:
# Check results diff between 1st pass and 2nd pass on same input (should be ~0)
[
    ((p1 - p0)**2).sum() for p1, p0 in zip(param_vals1, param_vals)
]

[tensor(0., device='cuda:0'),
 tensor(0., device='cuda:0'),
 tensor(0., device='cuda:0')]

In [15]:
# Top 5 tokens from LRP
lrp_answer_ids = input_ids[0][param_vals[1].flatten().topk(k=5).indices.cpu()]
print(tokenizer.decode(lrp_answer_ids))

 well under listed 150s


# SQuADv2 Evaluation

In [7]:
lrp.starting_relevance=None
lrp.topk=3

In [8]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

In [9]:
from tqdm import tqdm

results = []
top1_label_hits = 0
top1_model_hits = 0
total_examples = 0
total_intersect = 0
total_union = 0

for example in tqdm(dataset["validation"]):
    question = example["question"]
    context = example["context"]
    answers = example["answers"]["text"]

    if not answers:
        continue

    input_ids = tokenizer(context + " " + question, return_tensors="pt")["input_ids"].to(device)
    if input_ids.shape[-1] > 512:
        continue

    input_ids = tokenizer(question, context, return_tensors="pt")["input_ids"]
    output = model(input_ids.to(device))

    start = torch.argmax(output.start_logits[:,1:]) + 1
    end = torch.argmax(output.end_logits[:,1:]) + 2
    if start > end:
        continue
    model_answer = tokenizer.decode(input_ids[0][start:end], skip_special_tokens=True)
    if model_answer == "":
        continue
    # output = run_flan_encode_decode(input_ids)

    lrp.topk = end - start
    if (end - start) > output.start_logits.shape[-1]:
        print(start, end)
        print(output.start_logits.shape)
        break

    lrp_input1, lrp_input2 = output.start_logits, output.end_logits
    lrp_input1[0][0] = 0
    lrp_input2[0][0] = 0
    checkpoint_vals, param_vals = lrp.run((lrp_input1, lrp_input2))

    lrp_max = param_vals[1].flatten()[1:].argmax() + 1
    lrp_top_token = tokenizer.convert_ids_to_tokens([input_ids[0][lrp_max]])[0].strip().replace(chr(9601), "")

    # Do model answer-based accuracy, i.e. is the attribution aligned with the model prediction
    if start <= lrp_max <= end:
        top1_model_hits += 1

    # Do IoU with the model prediction
    intersect = 0
    union = lrp.topk
    for top_ind in param_vals[1].flatten()[1:].topk(lrp.topk).indices:
        top_ind = top_ind + 1
        if start <= top_ind <= end:
            intersect += 1
        else:
            union += 1
    total_intersect += intersect
    total_union += union

    # Do label-based accuracy
    if any(lrp_top_token in ans for ans in answers):
        # Is the attribution aligned with the ground truth label
        top1_label_hits += 1
    total_examples += 1
    if not (total_examples % 100):
        print(top1_model_hits, top1_label_hits, total_examples)
        print(total_intersect / total_union)
    # lrp_top5 = param_vals[2].flatten().topk(k=5)
    # lrp_top5_tokens = tokenizer.decode(input_ids[0][lrp_top5.indices.cpu()])
    # results.append({
    #     "example": example,
    #     "lrp_top5_tokens": lrp_top5_tokens,
    #     "lrp_top5_relevances": lrp_top5.values.cpu(),
    #     "is_impossible": len(answers) == 0
    # })

  2%|█▍                                                                            | 217/11873 [00:27<18:04, 10.74it/s]

81 23 100
tensor(0.5099, device='cuda:0')


  4%|██▊                                                                           | 429/11873 [00:52<20:07,  9.48it/s]

167 28 200
tensor(0.5025, device='cuda:0')


  5%|████▏                                                                         | 643/11873 [01:18<20:53,  8.96it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████▎ | 11603/11873 [34:35<01:18,  3.43it/s]

4753 821 5700
tensor(0.4653, device='cuda:0')


 99%|███████████████████████████████████████████████████████████████████████████▌| 11798/11873 [35:21<00:13,  5.67it/s]

4843 831 5800
tensor(0.4658, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████| 11873/11873 [35:40<00:00,  5.55it/s]


In [ ]:
from tqdm import tqdm

results = []

for example in tqdm(dataset["validation"].select(range(100))):
    question = example["question"]
    context = example["context"]
    answers = example["answers"]["text"]

    input_ids = tokenizer(question, context, return_tensors="pt")["input_ids"]
    output = model(input_ids.to(device))

    start = torch.argmax(output.start_logits)
    end = torch.argmax(output.end_logits) + 1
    model_answer = tokenizer.decode(input_ids[0][start:end], skip_special_tokens=True)

    checkpoint_vals, param_vals = lrp.run((output.start_logits, output.end_logits))

    lrp_top5 = param_vals[1].flatten().topk(k=5)
    lrp_top5_tokens = tokenizer.decode(input_ids[0][lrp_top5.indices.cpu()], skip_special_tokens=True)
    lrp_start_end = param_vals[1].flatten().topk(k=2).indices.cpu().sort()
    lrp_start = lrp_start_end[0][0]
    if lrp_start == 0:
        lrp_start = lrp_start_end[0][1]
    lrp_end = lrp_start_end[0][-1]
    lrp_answer_ids = input_ids[0][lrp_start:lrp_end + 1]
    lrp_answer = tokenizer.decode(lrp_answer_ids, skip_special_tokens=True)

    results.append({
        "example": example,
        "model_answer": model_answer,
        "lrp_answer": lrp_answer,
        "lrp_top5_tokens": lrp_top5_tokens,
        "lrp_top5_relevances": lrp_top5.values.cpu(),
        "is_impossible": len(answers) == 0
    })

In [ ]:
for res in results:
    print("Q: ", res["example"]["question"])
    print("A (labels): ", res["example"]["answers"]["text"])
    print("Model: ", res["model_answer"])
    print("LRP: ", res["lrp_answer"])
    print("LRP top5: ", res["lrp_top5_tokens"])
    print("LRP top5 attributions: ", res["lrp_top5_relevances"], '\n')